In [ ]:
!pip install -q mediapipe==0.10.0

In [ ]:
!wget -q https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/1/hand_landmarker.task

In [ ]:

from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np

import cv2
import copy
from google.colab.patches import cv2_imshow


MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

def draw_landmarks_on_image(rgb_image, detection_result):
  hand_landmarks_list = detection_result.hand_landmarks
  handedness_list = detection_result.handedness
  annotated_image = np.copy(rgb_image)

  # Loop through the detected hands to visualize.
  for idx in range(len(hand_landmarks_list)):
    hand_landmarks = hand_landmarks_list[idx]
    handedness = handedness_list[idx]

    # Draw the hand landmarks.
    hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    hand_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      hand_landmarks_proto,
      solutions.hands.HAND_CONNECTIONS,
      solutions.drawing_styles.get_default_hand_landmarks_style(),
      solutions.drawing_styles.get_default_hand_connections_style())

    # Get the top left corner of the detected hand's bounding box.
    height, width, _ = annotated_image.shape
    x_coordinates = [landmark.x for landmark in hand_landmarks]
    y_coordinates = [landmark.y for landmark in hand_landmarks]
    text_x = int(min(x_coordinates) * width)
    text_y = int(min(y_coordinates) * height) - MARGIN

    # Draw handedness (left or right hand) on the image.
    # cv2.putText(annotated_image, f"{handedness[0].category_name}",
    #             (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
    #             FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

  return annotated_image

def pre_process_landmark(landmark_list):
    temp_landmark_list = copy.deepcopy(landmark_list)

    # Convert to relative coordinates
    base_x, base_y = 0, 0
    for index, landmark_point in enumerate(temp_landmark_list):
        if index == 0:
            base_x, base_y = landmark_point[0], landmark_point[1]

        temp_landmark_list[index][0] = temp_landmark_list[index][0] - base_x
        temp_landmark_list[index][1] = temp_landmark_list[index][1] - base_y

    # Convert to a one-dimensional list
    temp_landmark_list = list(
        itertools.chain.from_iterable(temp_landmark_list))

    # Normalization
    max_value = max(list(map(abs, temp_landmark_list)))

    def normalize_(n):
        return n / max_value

    temp_landmark_list = list(map(normalize_, temp_landmark_list))

    return temp_landmark_list


def calc_landmark_list(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_point = []

    # Keypoint
    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)
        # landmark_z = landmark.z

        landmark_point.append([landmark_x, landmark_y])

    return landmark_point



In [ ]:
from google.colab import drive
drive.mount('/content/drive')
classesMap = {'0':'A +', '1':'B +','2':'C +', '3':'E +','4':'H +'}
DATA_DIR = '/content/drive/MyDrive/JestDiliBot_dataset/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import csv
import os

for letter in classesMap:
  csv_file_path = "/content/landmarks.csv"
  DATADIR = DATA_DIR+classesMap[letter]

  with open(csv_file_path, 'a', newline='') as csv_file:
      csv_writer = csv.writer(csv_file)

      for filename in os.listdir(DATADIR):
        if filename.endswith(".jpg"):
          # MediaPipe detector configuration
          base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
          options = vision.HandLandmarkerOptions(base_options=base_options, num_hands=1)
          detector = vision.HandLandmarker.create_from_options(options)


          image_path = os.path.join(DATADIR, filename)
          print("image_path: ", image_path)
          image = mp.Image.create_from_file(image_path)

          detection_result = detector.detect(image)
          coordinates = []
          for hand_index, hand_landmarks in enumerate(detection_result.hand_landmarks):
            for landmark_index, landmark in enumerate(hand_landmarks):
              # storing data
              row = [letter, int(landmark_index), float(landmark.x), float(landmark.y), landmark.visibility, landmark.presence]
              if landmark.presence == 1 or landmark.x == 0.0: continue
              coordinates.extend([float(landmark.x), float(landmark.y)])
          # writing into csv
          csv_writer.writerow([letter]+ coordinates)

          # showing image in console
          # annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)
          # annotated_image = cv2.resize(annotated_image, (256, 256))
          # cv2_imshow(cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))